In [ ]:
from gensim.models import word2vec, keyedvectors
import gensim.downloader as api
wv=api.load('word2vec-google-news-300')
vec_king=wv['king']

In [ ]:
import pandas as pd
messages=pd.read_csv('/content/drive/MyDrive/spam_ham.csv',sep='\t',names=['label','message'])
messages.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
vec_king.size
length=len(messages)
length

5572

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
import re
import nltk
corpus=[]
nltk.download('wordnet')
for i in range(0,length):
  review=re.sub('[^a-zA-Z]',' ',messages['message'][i])
  review=review.lower()
  review=review.split()
  review=[lemmatizer.lemmatize(word) for word in review]
  review=' '.join(review)
  corpus.append(review)



[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
len(corpus)

5572

In [ ]:
n = [(i, j, k) for i, j, k in zip(map(len, corpus), corpus, messages['message']) if i < 1]
n

[(0, '', '645'), (0, '', ':) '), (0, '', ':-) :-)')]

In [ ]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
# sent_tokenize(corpus[5571])
# simple_preprocess(corpus[5571])
words=[]
for sent in corpus:
  sent_token=sent_tokenize(sent)
  for sent in sent_token:
    words.append(simple_preprocess(sent))
len(words)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


5569

In [ ]:
import gensim

# Create an instance of the Word2Vec class
model = gensim.models.Word2Vec(sentences=words, vector_size=100, window=5, min_count=1, workers=4)

# Train the model
model.train(words, total_examples=len(words), epochs=5)

# Access the epochs attribute
print(model.epochs)  # Output: 10

5


In [ ]:
model.corpus_count

5569

In [ ]:
len(model.wv.index_to_key)

7199

In [ ]:
model.wv['is'].shape

(100,)

In [ ]:
words[0]

['go',
 'until',
 'jurong',
 'point',
 'crazy',
 'available',
 'only',
 'in',
 'bugis',
 'great',
 'world',
 'la',
 'buffet',
 'cine',
 'there',
 'got',
 'amore',
 'wat']

In [ ]:
import numpy as np
def avg_word2vec(doc):
  return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key],axis=0)


In [ ]:
!pip install tqdm

In [ ]:
from tqdm import tqdm
X=[]
for i in tqdm(range(len(words))):

  X.append(avg_word2vec(words[i]))

 76%|███████▌  | 4209/5569 [00:01<00:00, 2613.29it/s]/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 5569/5569 [00:02<00:00, 2478.56it/s]


In [ ]:
len(X)

5569

In [ ]:
    #dependent feature
y=messages[list(map(lambda x:len(x)>0,corpus))]
y=pd.get_dummies(y['label'])
y=y.iloc[:,0].values
y.shape

(5569,)

In [ ]:
count=0
for i in X:
  count+=1
  if i.shape!=(100,):

    print(count,i.shape)

4292 ()
5171 ()


In [ ]:
X[4291]=np.zeros(100)
X[5170]=np.zeros(100)

In [ ]:
X[4291]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
#independent features
x_new=np.array(X)
x_new.shape

(5569, 100)

In [ ]:
df=pd.DataFrame(x_new)
df.shape

(5569, 100)

In [ ]:
x=df


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_new,y,test_size=0.2,random_state=0)

In [ ]:
y.shape

(5569,)

In [ ]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((4455, 100), (1114, 100), (4455,), (1114,))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()
classifier.fit(x_train,y_train)

RandomForestClassifier()

In [ ]:
y_pred=classifier.predict(x_test)
y_pred

array([ True, False,  True, ...,  True,  True,  True])

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[139,  15],
       [ 11, 949]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9766606822262118

In [ ]:
from sklearn.metrics import precision_score,classification_report
print(precision_score(y_test,y_pred))
print(classification_report(y_test,y_pred))


0.9844398340248963
              precision    recall  f1-score   support

       False       0.93      0.90      0.91       154
        True       0.98      0.99      0.99       960

    accuracy                           0.98      1114
   macro avg       0.96      0.95      0.95      1114
weighted avg       0.98      0.98      0.98      1114



In [ ]:
import re
from nltk.stem import WordNetLemmatizer
from gensim.utils import simple_preprocess
from nltk import sent_tokenize
import numpy as np

def preprocess_text(text):
  review = re.sub('[^a-zA-Z]', ' ', text)
  review = review.lower()
  review = review.split()
  lemmatizer = WordNetLemmatizer()
  review = [lemmatizer.lemmatize(word) for word in review]
  review = ' '.join(review)
  words = []
  for sent in sent_tokenize(review):
    words.extend(simple_preprocess(sent))  # Use extend instead of append
  return words

def predict_text_class(text, classifier, model):
  words = preprocess_text(text)
  feature_vector = avg_word2vec(words, model)
  if feature_vector.shape != (100,):
    feature_vector = np.zeros(100)
  prediction = classifier.predict([feature_vector])[0]  # Wrap feature_vector in a list
  return prediction

def avg_word2vec(doc, model):
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key], axis=0)


# Assuming 'classifier' and 'model' are your trained classifier and Word2Vec model
text = "abb ke bolu"
prediction = predict_text_class(text, classifier, model)

# Print the prediction (0 for ham, 1 for spam)
print(prediction)


In [ ]:
model.save("word2vec_model.model")

In [ ]:
pip install streamlit gensim scikit-learn nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.8 MB/s eta 0:00:00


In [ ]:
import pickle
with open("random_forest_classifier.pkl", "wb") as f:
    pickle.dump(classifier, f)

In [ ]:

%%writefile app.py
import streamlit as st
import re
from nltk.stem import WordNetLemmatizer
from gensim.utils import simple_preprocess
from nltk import sent_tokenize
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from gensim.models import Word2Vec
import pickle  # Save and load trained models

# Load your trained model and classifier
model = Word2Vec.load("word2vec_model.model")  # Replace with your saved Word2Vec model path
with open("random_forest_classifier.pkl", "rb") as f:  # Replace with your saved classifier path
    classifier = pickle.load(f)

# Text preprocessing function
def preprocess_text(text):
    review = re.sub('[^a-zA-Z]', ' ', text)
    review = review.lower()
    review = review.split()
    lemmatizer = WordNetLemmatizer()
    review = [lemmatizer.lemmatize(word) for word in review]
    review = ' '.join(review)
    words = []
    for sent in sent_tokenize(review):
        words.extend(simple_preprocess(sent))
    return words

# Average Word2Vec calculation
def avg_word2vec(doc, model):
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key], axis=0)

# Prediction function
def predict_text_class(text, classifier, model):
    words = preprocess_text(text)
    feature_vector = avg_word2vec(words, model)
    if feature_vector is None or feature_vector.shape != (100,):
        feature_vector = np.zeros(100)
    prediction = classifier.predict([feature_vector])[0]
    return "Spam" if prediction == 1 else "Ham"

# Streamlit UI
st.title("Spam Message Classifier")
st.write("Enter a message to classify it as Ham or Spam.")

# Input message
user_message = st.text_input("Message", placeholder="Type your message here...")

# Predict button
if st.button("Classify"):
    if user_message.strip():
        result = predict_text_class(user_message, classifier, model)
        st.subheader(f"The message is classified as: {result}")
    else:
        st.error("Please enter a message to classify!")

Writing app.py


In [ ]:
# Save the trained classifier
with open('random_forest_classifier.pkl', 'wb') as f:
    pickle.dump(classifier, f)

In [ ]:
!pip install streamlit -q


In [ ]:
!wget -q -O - ipv4.icanhazip.com

104.199.187.231


In [ ]:
!streamlit run app.py &npx localtunnel --port 8501

⠙

⠹⠸⠼⠴⠦
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://104.199.187.231:8501

⠧your url is: https://late-pandas-jump.loca.lt
